In [14]:
import pandas as pd
from utils import *

In [25]:
excel_input = 'output/output.xlsx'

excel_file = pd.ExcelFile(excel_input)

In [3]:
df = pd.read_excel(excel_input, sheet_name='Abrasives_raw_info')

In [6]:
unnamed_cols = [col for col in df.columns if 'Unnamed' in col]
split_indices = [df.columns.get_loc(col) for col in unnamed_cols]

# Ensure the indices are sorted and unique
split_indices = sorted(set(split_indices))

# Function to split the DataFrame at the specified column indices
def split_dataframe(df, split_indices):
    dfs = []
    start_col = 0
    for index in split_indices:
        dfs.append(df.iloc[:, start_col:index])
        start_col = index + 1
    # Append the last segment
    dfs.append(df.iloc[:, start_col:])
    return dfs

# Split the DataFrame
dfs = split_dataframe(df, split_indices)

In [15]:
for df in dfs:
  # si il existe une case ou il est écrit 'World total (rounded)' ou 'World total', ou n'imprt quele chose proche a ceci, crée une list new_df et append cette dataframe là dessus
  target_phrases = ['World total (rounded)', 'World total']

# Create a list to hold the dataframes that meet the condition
  new_df = []

  # Iterate through the split dataframes and check the condition
  for df in dfs:
      # Check if any cell in the dataframe matches the target phrases
      if df.apply(lambda row: any(match_element_in_text(target_phrases, str(cell)) for cell in row), axis=1).any():
          new_df.append(df)

In [19]:
new_df[0].head(4)

,text.1,text_1.1,text_2.1,text_3.1,text_4.1
0,NaN,Fused aluminum oxidee,Fused aluminum oxidee,Silicon carbidee,Silicon carbidee
1,NaN,2022,2023,2022,2023
2,United States,—,—,"40,000","40,000"
3,United States and Canada,"60,000","60,000",—,—


In [34]:
len(excel_file.sheet_names)  # 65 table!

82

: 

In [29]:
sheet_names = excel_file.sheet_names

# Initialize remain_df as an empty list
remain_df = []

for name in sheet_names:
    df = pd.read_excel(excel_input, sheet_name=name)
    
    unnamed_cols = [col for col in df.columns if 'Unnamed' in col]
    split_indices = [df.columns.get_loc(col) for col in unnamed_cols]

    # Ensure the indices are sorted and unique
    split_indices = sorted(set(split_indices))

    # Split the DataFrame
    dfs = split_dataframe(df, split_indices)

    # List to hold the dataframes that meet the condition
    new_df = []

    # Target phrases
    target_phrases = ['World total (rounded)', 'World total']

    # Iterate through the split dataframes and check the condition
    for df in dfs:
        # Check if any cell in the dataframe matches the target phrases
        if df.apply(lambda row: any(match_element_in_text(target_phrases, str(cell)) for cell in row), axis=1).any():
            new_df.append(df)

    # Extend remain_df with new_df
    remain_df.extend(new_df)

In [32]:
remain_df

[                      text.1               text_1.1               text_2.1  \
 0                        NaN  Fused aluminum oxidee  Fused aluminum oxidee   
 1                        NaN                   2022                   2023   
 2              United States                      —                      —   
 3   United States and Canada                 60,000                 60,000   
 4                  Australia                 50,000                 50,000   
 5                    Austria                 90,000                 90,000   
 6                     Brazil                 50,000                 50,000   
 7                      China                800,000                800,000   
 8                     France                 40,000                 40,000   
 9                    Germany                 80,000                 80,000   
 10                     India                 40,000                 40,000   
 11                     Japan                 15,000

In [33]:
def save_dfs_to_excel(dfs, filename):
    with pd.ExcelWriter(filename) as writer:
        for i, df in enumerate(dfs):
            df.to_excel(writer, sheet_name=f'Sheet{i+1}', index=False)

# Save the DataFrames to an Excel file
save_dfs_to_excel(remain_df, 'needed_tables.xlsx')